# Mnist Dataloader
https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook

In [1]:
import numpy as np # linear algebra
import struct
from array import array
from os.path import join

# MNIST Data Loader Class
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        # print(images)
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return x_train, y_train, x_test, y_test

In [2]:
# Set file paths based on added MNIST Datasets
input_path = r'./mnist/'
training_images_filepath = join(input_path, 'train-images.idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels.idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images.idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels.idx1-ubyte')

# Load MINST dataset
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
train_img, train_lbs, test_img, test_lbs = mnist_dataloader.load_data()

In [3]:
type(train_img[0][0])

numpy.ndarray

# Leitura mnist utilizando 'with open'

In [4]:
with open(training_images_filepath, 'rb') as i:
    magic, size = struct.unpack('>II', i.read(8))
    train_img = np.fromfile(i, dtype=np.dtype(np.uint8)).newbyteorder(">")  

with open(training_labels_filepath, 'rb') as i:
    magic, size = struct.unpack('>II', i.read(8))
    train_lbs = np.fromfile(i, dtype=np.dtype(np.uint8)).newbyteorder(">")

with open(test_images_filepath, 'rb') as i:
    magic, size = struct.unpack('>II', i.read(8))
    test_img = np.fromfile(i, dtype=np.dtype(np.uint8)).newbyteorder(">")

with open(test_labels_filepath, 'rb') as i:
    magic, size = struct.unpack('>II', i.read(8))
    test_lbs = np.fromfile(i, dtype=np.dtype(np.uint8)).newbyteorder(">")

# Leitura utilizando idx2numpy

In [5]:
import numpy as np
import idx2numpy

train_img = idx2numpy.convert_from_file(r"./mnist/train-images.idx3-ubyte").reshape(60000,1,28*28)
train_lbs = idx2numpy.convert_from_file(r"./mnist/train-labels.idx1-ubyte").reshape(-1,1)

test_img = idx2numpy.convert_from_file(r"./mnist/t10k-images.idx3-ubyte").reshape(10000,1,28*28)
test_lbs = idx2numpy.convert_from_file(r"./mnist/t10k-labels.idx1-ubyte").reshape(-1,1)

print(train_img.shape)
print(train_lbs.shape)
print(test_img.shape)
print(test_lbs.shape)

(60000, 1, 784)
(60000, 1)
(10000, 1, 784)
(10000, 1)


# Normalização dos valores entre [0,1]

In [6]:
train_img = train_img.astype("float32") / 255
test_img = test_img.astype("float32") / 255

# One Hot Encoding Labels

In [7]:
def OHE(y_lbs:np.ndarray,num_classes:int):
    ohs = np.zeros((y_lbs.shape[0],num_classes))
    for i,j in enumerate(y_lbs):
        ohs[i][j] = 1
    return ohs

train_lbs = OHE(train_lbs,10)
test_lbs = OHE(test_lbs,10)

# Implementação da rede

In [8]:
# import numpy as np

from network import Network
from fc_layer import FCLayer
from activation_layer import ActivationLayer
from MiniBatchGD import mini_batch
from activations import (
    tanh,
    tanh_prime,
    sigmoid,
    sigmoid_prime,
    ReLU,
    ReLU_prime,
    leakyReLU,
    leakyReLU_prime,
    linear,
    linear_prime,
    softmax,
    softmax_prime
)
from losses import mse, mse_prime
from optimizers import adam, sgd
from weights import (
    he_normal,
    he_uniform,
    normal,
    uniform,
    xavier_normal,
    xavier_uniform,
)

In [9]:
net = Network()
net.add(FCLayer(28 * 28, 100))
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.add(FCLayer(100, 50))
net.add(ActivationLayer(leakyReLU, leakyReLU_prime))
net.add(FCLayer(50, 10))
net.add(ActivationLayer(ReLU, ReLU_prime))

# train
net.use(mse, mse_prime)
net.fit(train_img, train_lbs, epochs=5, batch_size=64, optimizer=adam(),verbose=True)

# test
out = net.predict(test_img)


epoch:1    iteration:1/938   error=7.785210496495484    Acc:0.0
epoch:1    iteration:2/938   error=6.156294042167613    Acc:0.0
epoch:1    iteration:3/938   error=6.336885452991898    Acc:0.03125
epoch:1    iteration:4/938   error=6.212335619418335    Acc:0.015625
epoch:1    iteration:5/938   error=6.29166938343238    Acc:0.0
epoch:1    iteration:6/938   error=6.392830360163597    Acc:0.03125
epoch:1    iteration:7/938   error=6.33356344854265    Acc:0.0625
epoch:1    iteration:8/938   error=6.161129618648898    Acc:0.0
epoch:1    iteration:9/938   error=6.067217406635672    Acc:0.0
epoch:1    iteration:10/938   error=6.250983827410421    Acc:0.03125
epoch:1    iteration:11/938   error=6.249279203463668    Acc:0.0
epoch:1    iteration:12/938   error=6.168939109459278    Acc:0.0
epoch:1    iteration:13/938   error=6.2285793429101375    Acc:0.0
epoch:1    iteration:14/938   error=5.936922465617317    Acc:0.140625
epoch:1    iteration:15/938   error=5.890825610929322    Acc:0.234375
epoch

In [10]:
for i in range(len(test_lbs[:5])):
    print("y    :",np.argmax(test_lbs[i]))
    print("y_hat:",np.argmax(out[i]))
    print()

y    : 7
y_hat: 0

y    : 2
y_hat: 5

y    : 1
y_hat: 1

y    : 0
y_hat: 0

y    : 4
y_hat: 4



# Acurácia

In [11]:
from metrics import accuracy

acc = accuracy(test_lbs,out)
print(f"Accuracy: {acc}")

Accuracy: 0.7483
